# mBuild Tutorial 05: Creating Flexible Classes

This tutorial demonstrates how to write flexible classes that can be used to create families of `Compounds`.  In particular, the previous class we wrote to create butane is extended to allow construction of linear alkanes of arbitrary chain length.

As in the prior tutorials, we need to first import mbuild (here as `mb`).

We also introduce a filter for somewarnings to provide cleaner output.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import mbuild as mb

In the previous few tutorials we found that we could create reusable `Compounds` by wrapping routines for creating particles and bonds into a class. However, if we have to create a new class for each molecule we want to examine this would still be quite cumbersome, particularly if we desired to screen over a large chemical parameter space. Fortunately, this problem is easily solved by including additional arguments with the class constructor. In this way, one or more top-level variables describing the molecular chemistry can be  used to create a whole family of molecules. 

We'll demonstrate that here by modifying the Butane class, defined in the previous tutorial, to now allow for the creation of any linear alkane by adding a `chain_length` argument.

Note, rather than defining the CH2 and H classes again, we will use the classes for these that are included in the mBuild library.

In [2]:
from mbuild.lib.atoms import H
from mbuild.lib.moieties import CH2

class Alkane(mb.Compound):
    def __init__(self, chain_length):
        super(Alkane, self).__init__()
        
        hydrogen = H()
        last_unit = CH2()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_unit['up'])
        self.add(last_unit, label='ch2[$]')
        self.add(hydrogen, label='up-cap')
        for _ in range(chain_length - 1):
            current_unit = CH2()
            mb.force_overlap(move_this=current_unit,
                             from_positions=current_unit['up'],
                             to_positions=last_unit['down'])
            self.add(current_unit, label='ch2[$]')
            last_unit=current_unit
        hydrogen = H()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_unit['down'])
        self.add(hydrogen, label='down-cap')

We can now create any linear alkane by simply providing a different value for `chain_length` upon instantiation.

In [3]:
ethane = Alkane(chain_length=2)
ethane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [4]:
hexane = Alkane(chain_length=6)
hexane.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Creation of an imidazole with an arbitrary side chain length

The linear alkane class above can be trivially modified to create more complex molecules, e.g. by changing the chemistry of one or both of the capping groups. Here, we will replace one of the hydrogen caps with an imidazole ring. 

Let us first start by creating an `Imidazole` class, loading in the structure from a MOL2 file, and adding a `Port` to the appropriate location for the tail.

In [5]:
class Imidazole(mb.Compound):
    def __init__(self):
        super(Imidazole, self).__init__()
        mb.load('utils/cmim.mol2', compound=self)
        #define C_tail to be the N atom for which the carbon tail will attach. 
        #This is the 4th entry in the datafile provided, hence self[3]
        C_tail = self[3] 
        #add a port
        self.add(mb.Port(anchor=C_tail, orientation=[0, 1, 0], separation=0.04), 'up')

Next we will create a CMIM class that largely mimics the alkane class, but changes one capping hydrogen for an Imidazole. 

In [6]:
from mbuild.lib.atoms import H
from mbuild.lib.moieties import CH2

class CMIM(mb.Compound):
    def __init__(self, chain_length):
        super(CMIM, self).__init__()
        
        hydrogen = H()
        last_unit = CH2()
        mb.force_overlap(move_this=hydrogen,
                         from_positions=hydrogen['up'],
                         to_positions=last_unit['up'])
        self.add(last_unit, label='ch2[$]')
        self.add(hydrogen, label='up-cap')
        for _ in range(chain_length - 1):
            current_unit = CH2()
            mb.force_overlap(move_this=current_unit,
                             from_positions=current_unit['up'],
                             to_positions=last_unit['down'])
            self.add(current_unit, label='ch2[$]')
            last_unit=current_unit
        imidazole = Imidazole()
        mb.force_overlap(move_this=imidazole,
                         from_positions=imidazole['up'],
                         to_positions=last_unit['down'])
        self.add(imidazole, label='down-cap')

Here, we can use this to create 1-Butyl-3-methylimidazolium (bmim), by passing a side chain length of 4.

In [7]:
bmim = CMIM(chain_length=4)
bmim.visualize()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Recap

The goal of this tutorial was to demonstrate how to create `Compound` classes with arguments that can be used to tune the molecular chemistry.

The next tutorial will teach you how to create systems of bulk molecules.